In [1]:
#@title read all person ids { form-width: "123px" }

import pandas as pd

def full_path(name):
  return f"https://github.com/LBNL-UCB-STI/beam/raw/Xuan/Oct-EPA-Parking-Zone-Link-Level/test/input/sfbay/ActivitySim/{name}"

file_name = 'persons.csv.gz'
file_path = full_path(file_name)
all_persons = pd.concat([df[['person_id','worker','earning']] for df in pd.read_csv(file_path, low_memory=False, chunksize=10000)])
all_persons = all_persons[all_persons.worker == 0]
all_persons_ids = set(all_persons['person_id'].unique())
print(f'there are {len(all_persons_ids)} persons')

there are 3471394 persons


In [2]:
all_persons

,person_id,worker,earning
0,0,0,0
3,379,0,0
4,380,0,800
6,663,0,0
7,664,0,0
...,...,...,...
6741448,6739911,0,0
6741450,6740089,0,400
6741455,6740670,0,0
6741457,6740836,0,0


In [3]:
#@title sample household ids based on sampled persons { form-width: "123px" }

from random import sample

randomly_selected_persons_ids = set(all_persons_ids)

file_name = 'persons.csv.gz'
file_path = full_path(file_name)
randomly_selected_persons = pd.concat([df[df['person_id'].isin(randomly_selected_persons_ids)][['person_id','household_id']] 
                                       for df in pd.read_csv(file_path, low_memory=False, chunksize=10000)])

selected_households_ids = set(randomly_selected_persons['household_id'].unique())
print(f'there are {len(selected_households_ids)} selected households')

there are 1818972 selected households


In [4]:
#@title sample the whole input based on selected households { form-width: "123px" }

file_name = 'persons.csv.gz'
file_path = full_path(file_name)
selected_persons = pd.concat([df[df['household_id'].isin(selected_households_ids)] for df in pd.read_csv(file_path, low_memory=False, chunksize=10000)])
print(f'for {file_name} shape is {selected_persons.shape}')

selected_persons_ids = set(selected_persons['person_id'].unique())
print(f'there are {len(selected_persons_ids)} selected persons')

file_name = 'households.csv.gz'
file_path = full_path(file_name)
selected_households = pd.concat([df[df['household_id'].isin(selected_households_ids)] for df in pd.read_csv(file_path, low_memory=False, chunksize=10000)])
print(f'for {file_name} shape is {selected_households.shape}')

file_name = 'plans.csv.gz'
file_path = full_path(file_name)
selected_plans = pd.concat([df[df['person_id'].isin(selected_persons_ids)] for df in pd.read_csv(file_path, low_memory=False, chunksize=10000)])
print(f'for {file_name} shape is {selected_plans.shape}')

file_name = 'trips.csv.gz'
file_path = full_path(file_name)
selected_trips = pd.concat([df[df['person_id'].isin(selected_persons_ids)] for df in pd.read_csv(file_path, low_memory=False, chunksize=10000)])
print(f'for {file_name} shape is {selected_trips.shape}')

for persons.csv.gz shape is (5453958, 13)
there are 5453958 selected persons
for households.csv.gz shape is (1818972, 12)
for plans.csv.gz shape is (41952754, 11)
for trips.csv.gz shape is (18573587, 15)


In [5]:
selected_persons.to_csv('persons.csv.gz', compression='gzip')
selected_households.to_csv('households.csv.gz', compression='gzip')
selected_plans.to_csv('plans.csv.gz', compression='gzip')
selected_trips.to_csv('trips.csv.gz', compression='gzip')
!ls -lah

total 2758968
drwxr-xr-x@  8 jiangxuan  staff   256B Oct 24 23:18 .
drwxr-xr-x@ 13 jiangxuan  staff   416B Oct 24 23:18 ..
drwxr-xr-x@  3 jiangxuan  staff    96B Oct 24 21:09 .ipynb_checkpoints
-rw-r--r--   1 jiangxuan  staff    11K Oct 24 23:07 forDownSample.ipynb
-rw-r--r--   1 jiangxuan  staff    34M Oct 24 23:08 households.csv.gz
-rw-r--r--   1 jiangxuan  staff    68M Oct 24 23:07 persons.csv.gz
-rw-r--r--   1 jiangxuan  staff   701M Oct 24 23:16 plans.csv.gz
-rw-r--r--   1 jiangxuan  staff   521M Oct 24 23:20 trips.csv.gz


In [6]:
out_plans_url = 'https://beam-outputs.s3.amazonaws.com/output/sfbay/sfbay-AS-60k-charging-1__2021-06-14_12-50-23_ebd/plans.csv.gz'
out_plans = pd.read_csv(out_plans_url)
display(out_plans.head(3))

unique_person_ids = set(out_plans['personId'].unique())
print(f'there are {len(unique_person_ids)} unique person ids')

,personId,planIndex,planScore,planSelected,planElementType,planElementIndex,activityType,activityLocationX,activityLocationY,activityEndTime,legMode,legDepartureTime,legTravelTime,legRouteType,legRouteStartLink,legRouteEndLink,legRouteTravelTime,legRouteDistance,legRouteLinks
0,4116,0,0.0,True,activity,0,Home,566306.532932,4.189607e+06,22240.8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,4116,0,0.0,True,leg,1,NaN,NaN,NaN,NaN,car,-inf,-inf,NaN,NaN,NaN,NaN,NaN,NaN
2,4116,0,0.0,True,activity,2,work,565989.607950,4.189081e+06,51991.2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


there are 60150 unique person ids


In [7]:
file_name = 'persons.csv.gz'
file_path = f'https://activity-sim-outputs-for-beam.s3.us-east-2.amazonaws.com/activity-sim-output-base-2010/{file_name}'
selected_persons = pd.concat([df[df['person_id'].isin(unique_person_ids)] for df in pd.read_csv(file_path, low_memory=False, chunksize=10000)])
print(f'for {file_name} shape is {selected_persons.shape}')

unique_household_ids = set(selected_persons['household_id'].unique())
print(f"there are {len(unique_household_ids)} unique houlsehold ids")

file_name = 'households.csv.gz'
file_path = f'https://activity-sim-outputs-for-beam.s3.us-east-2.amazonaws.com/activity-sim-output-base-2010/{file_name}'
selected_households = pd.concat([df[df['household_id'].isin(unique_household_ids)] for df in pd.read_csv(file_path, low_memory=False, chunksize=10000)])
print(f'for {file_name} shape is {selected_households.shape}')

file_name = 'plans.csv.gz'
file_path = f'https://activity-sim-outputs-for-beam.s3.us-east-2.amazonaws.com/activity-sim-output-base-2010/{file_name}'
selected_plans = pd.concat([df[df['person_id'].isin(unique_person_ids)] for df in pd.read_csv(file_path, low_memory=False, chunksize=10000)])
print(f'for {file_name} shape is {selected_plans.shape}')

file_name = 'trips.csv.gz'
file_path = f'https://activity-sim-outputs-for-beam.s3.us-east-2.amazonaws.com/activity-sim-output-base-2010/{file_name}'
selected_trips = pd.concat([df[df['person_id'].isin(unique_person_ids)] for df in pd.read_csv(file_path, low_memory=False, chunksize=10000)])
print(f'for {file_name} shape is {selected_trips.shape}')

for persons.csv.gz shape is (60150, 13)
there are 25313 unique houlsehold ids
for households.csv.gz shape is (25313, 12)
for plans.csv.gz shape is (532196, 11)
for trips.csv.gz shape is (236023, 15)


In [8]:
selected_persons.to_csv('60k_persons.csv.gz', compression='gzip')
selected_households.to_csv('60k_households.csv.gz', compression='gzip')
selected_plans.to_csv('60k_plans.csv.gz', compression='gzip')
selected_trips.to_csv('60k_trips.csv.gz', compression='gzip')
!ls -lah

total 2796912
drwxr-xr-x@ 13 jiangxuan  staff   416B Oct 24 23:29 .
drwxr-xr-x@ 13 jiangxuan  staff   416B Oct 24 23:28 ..
drwxr-xr-x@  4 jiangxuan  staff   128B Oct 24 23:24 .ipynb_checkpoints
-rw-r--r--   1 jiangxuan  staff   573K Oct 24 23:29 60k_households.csv.gz
-rw-r--r--   1 jiangxuan  staff   869K Oct 24 23:29 60k_persons.csv.gz
-rw-r--r--   1 jiangxuan  staff   9.0M Oct 24 23:29 60k_plans.csv.gz
-rw-r--r--   1 jiangxuan  staff   6.9M Oct 24 23:29 60k_trips.csv.gz
-rw-r--r--   1 jiangxuan  staff    18K Oct 24 23:27 forDownSample.ipynb
-rw-r--r--   1 jiangxuan  staff    34M Oct 24 23:08 households.csv.gz
-rw-r--r--   1 jiangxuan  staff    68M Oct 24 23:07 persons.csv.gz
-rw-r--r--   1 jiangxuan  staff   701M Oct 24 23:16 plans.csv.gz
-rw-r--r--   1 jiangxuan  staff    10K Oct 24 23:26 read_files.ipynb
-rw-r--r--   1 jiangxuan  staff   521M Oct 24 23:20 trips.csv.gz
